# Merge Images and JSONs

In [ ]:
import sys
sys.path.insert(0,'..')  # add project root to PATH

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import cv2

In [ ]:
from src.preprocessing import JSONUtil, fix_annotation_keys

In [ ]:
# define input and output path for images
input_path = "../data/"
output_path = "../_data/"

## 1. Load, merge and fix annotations

In [ ]:
def correct_annotations_dict(annotations_dict):
    for image_name, annotation in annotations_dict.items():
        # fix region_attributes: rename 'Object' to 'Objects'
        for region in annotation['regions']:
            if "Object" in region["region_attributes"]:
                print("Mistake detected and fixed ", image_name)
                region["region_attributes"]["Objects"] = region["region_attributes"].pop("Object")
    
        # output images without bounding boxes
        if len(annotation['regions']) == 0:
            print("No boxes in :", image_name)
    return annotations_dict

In [ ]:
annotations_dict = fix_annotation_keys(JSONUtil.merge_jsons(input_path))
annotations_dict = correct_annotations_dict(annotations_dict)

## 2 Load all images from the input directory and write them to the output 

In [ ]:
# loop through all images in input folder and save in output
for image_file in pathlib.Path(input_path).rglob('*.jpg'):
    # read image
    image = cv2.imread(str(image_file))
    # write image
    image_out_file = pathlib.Path(output_path, image_file.name)
    cv2.imwrite(str(image_out_file), image)

In [ ]:
final_json_files = annotations_dict.copy()
# loop through json and check existance of all images
for filename in annotations_dict:
    # test if read image is working
    test = cv2.imread(str(pathlib.Path(output_path, filename)))
    if test is None:
        print("Image not found:", filename)
        del final_json_files[filename]

In [ ]:
# dump all labeling information into master .json file
JSONUtil.write(pathlib.Path(output_path, "Master.json"), final_json_files)